In [ ]:
import pandas as pd
import json

# Load the JSON data (replace 'your_file.json' with your file path)
with open('data.json', 'r') as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.DataFrame(data)

# Extract labels from the 'annotations' column
# This assumes that each 'text' has a list of annotations
labels = []
values = []
for row in df['annotations']:
  if not row:
    labels.append("NULL")
    values.append("")
  for annotation in row:
      labels.append(annotation['label'])
      values.append(annotation['entity'])

# Convert labels to a DataFrame and count occurrences
labels_df = pd.DataFrame({'Labels': labels, 'Entity': values})
# label_counts = labels_df['label'].value_counts()

# Display the count of each label
# print(label_counts)


In [ ]:
def summarize_data(df):
  labels = []
  values = []
  for row in df['annotations']:
    if not row:
      labels.append("NULL")
      values.append("")
    for annotation in row:
      labels.append(annotation['label'])
      values.append(annotation['entity'])
  labels_df = pd.DataFrame({'Labels': labels, 'Entity': values})
  return labels_df

In [ ]:
labels_df

,Labels,Entity
0,DOMAIN_NAME,Escola Digital
1,GEO_LOCATION,Brazilian
2,DOMAIN_NAME,MyTopDog
3,GEO_LOCATION,South African
4,DOMAIN_NAME,Okoo
...,...,...
160,MALWARE_NAME,SpyEye Trojan
161,GEO_LOCATION,U.S.
162,MALWARE_NAME,Win32/Gapz
163,MALWARE_NAME,Win32/Gapz


In [ ]:
labels_df.groupby([labels]).count()

,Labels,Entity
ACTOR,21,21
DOMAIN_NAME,22,22
FILE_PATH,5,5
GEO_LOCATION,38,38
HASH_VAL,6,6
IP,5,5
MALWARE_NAME,50,50
NULL,10,10
URL,8,8


In [ ]:
df

,text,annotations
0,"Escola Digital, a Brazilian website offering a...","[{'start': 0, 'end': 13, 'label': 'DOMAIN_NAME..."
1,"Over the past several weeks, a number of banks...","[{'start': 168, 'end': 181, 'label': 'URL', 'e..."
2,The JCPA website's homepage is infected with a...,"[{'start': 164, 'end': 197, 'label': 'DOMAIN_N..."
3,"Ultimately, the exploits are served from cdn3[...","[{'start': 41, 'end': 106, 'label': 'URL', 'en..."
4,This exploit kit served two (Qbot) binaries wi...,"[{'start': 60, 'end': 91, 'label': 'HASH_VAL',..."
...,...,...
113,"... In another report, SMS Android Trojan wa...","[{'start': 24, 'end': 42, 'label': 'MALWARE_NA..."
114,... to hackers with malicious intent. This inc...,"[{'start': 197, 'end': 210, 'label': 'MALWARE_..."
115,"... smartphones, has come of age and is increa...","[{'start': 219, 'end': 223, 'label': 'GEO_LOCA..."
116,... 2012 - 02:00AM Win32/Gapz new bootkit tech...,"[{'start': 19, 'end': 29, 'label': 'MALWARE_NA..."


In [ ]:
!pip install -U spacy

!python -m spacy download en_core_web_sm

import json


train_file = "data.json"

def load_data(data_path=train_file):
  with open(train_file, 'r') as json_file:
      data = json.load(json_file)
  return data

def extract_entities(annotation):
    start = annotation['start']
    end = annotation['end']
    label = annotation['label'].upper()
    return start, end, label

def process_example(example):
    entities = [extract_entities(annotation) for annotation in example['annotations']]
    return {
        'text': example['text'],
        'entities': entities
    }

data = load_data()
training_data = [process_example(example) for example in data]

2023-12-03 07:27:49.808866: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 07:27:49.808921: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 07:27:49.808952: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 07:27:51.043850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 84.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
training_data[0]

{'text': 'Escola Digital, a Brazilian website offering a wide range of digital courses for both students and teachers was found leaking the personal records of nearly 75,000 active users between 2016 and 2017. MyTopDog, a South African children-oriented study platform providing practice tests and interactive games, exposed over 800,000 student records, courtesy of a misconfigured Amazon S3 bucket. Okoo, an online learning platform for children in Kazakhstan, exposed 7,200 user records that included PII and nearly 1 million entries regarding user activity on the platform and analytics. Square Panda, a US-based virtual platform that helps children learn how to read and write, exposed the information of nearly 15,000 users. Playground Sessions, a virtual piano lesson platform in the United States, revealed the private information of around 4,100 users registered between 2011 and 2013.',
 'entities': [(0, 13, 'DOMAIN_NAME'),
  (18, 26, 'GEO_LOCATION'),
  (200, 207, 'DOMAIN_NAME'),
  (212, 

In [ ]:
from typing_extensions import DefaultDict
import random
def split_data_train_dev(data,percentage=0.8):
  label_to_idx = DefaultDict(set)
  training_idx = []
  testing_idx = []
  for i in range(len(data)):
    for entity in data[i]['entities']:
      label_to_idx[entity[2]].add(i)

  print('label_to_idx ****', label_to_idx)
  for label, idx in label_to_idx.items():
    num_data_points = len(idx)
    num_train_samples = round(num_data_points * percentage)
    idx_lst = list(idx)
    random.Random(4).shuffle(idx_lst)
    train_indices = idx_lst[:num_train_samples]
    test_indices = idx_lst[num_train_samples:]
    training_idx.extend(train_indices)
    testing_idx.extend(test_indices)

  training_data = [data[i] for i in training_idx]
  testing_data = [data[i] for i in testing_idx]

  return training_data, testing_data

def process_training_example(nlp, training_example):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    entities = []

    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
          span1 = doc.char_span(start, end+1, label=label, alignment_mode="contract")
          if span1 is None:
            print(f"Skipping invalid entity: {label} [{start}:{end}]")
          else:
            entities.append(span1)
        else:
            entities.append(span)

    filtered_entities = spacy.util.filter_spans(entities)
    doc.ents = filtered_entities
    return doc


In [ ]:
train, test = split_data_train_dev(training_data)

label_to_idx **** defaultdict(<class 'set'>, {'DOMAIN_NAME': {0, 2, 38, 71, 73, 74, 23, 24}, 'GEO_LOCATION': {0, 5, 6, 9, 11, 15, 36, 49, 50, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 82, 93, 100, 115}, 'URL': {32, 1, 96, 3, 14, 80, 86, 26}, 'MALWARE_NAME': {3, 4, 7, 10, 11, 12, 13, 17, 22, 27, 35, 39, 41, 42, 43, 44, 45, 46, 48, 53, 54, 55, 76, 77, 78, 79, 81, 83, 99, 101, 102, 103, 104, 105, 106, 109, 112, 113, 114, 116, 117}, 'HASH_VAL': {25, 34, 4, 20}, 'ACTOR': {33, 37, 8, 47, 92, 18, 51, 52, 21, 84, 85, 87, 89, 94, 91, 28, 30, 31}, 'FILE_PATH': {16, 40, 107}, 'IP': {19, 29}})


In [ ]:
train[0]

{'text': 'The only thing I can say for sure is that China does have a huge problem with media pirating and seems to do nothing or not much about it.  OVGUIDE.COM  will link you to thousands of streaming movies\\u2026.most of which are hosted on Chinese sites.',
 'entities': [(140, 151, 'DOMAIN_NAME')]}

In [ ]:
len(train)

88

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

def convert_to_spacy(training_data, output_path, desc="Processing training data"):
  # Load a new spaCy model
  nlp = spacy.blank("en")
  # Create a DocBin to store processed documents
  doc_bin = DocBin()
  # Process training data and add to DocBin
  for training_example in tqdm(training_data, desc=desc):
      processed_doc = process_training_example(nlp, training_example)
      doc_bin.add(processed_doc)

  # Save the processed documents to disk
  doc_bin.to_disk(output_path)


In [ ]:

convert_to_spacy(train, "train.spacy")
convert_to_spacy(test, "dev.spacy", "Processing testing data")

Processing training data: 100%|██████████| 88/88 [00:00<00:00, 939.01it/s]


Skipping invalid entity: HASH_VAL [197:262]


Processing testing data: 100%|██████████| 24/24 [00:00<00:00, 579.16it/s]


In [ ]:
len(test)

24

In [ ]:
#initialize config
!python -m spacy init fill-config base_config.cfg base_config.cfg

#train
!python -m spacy train base_config.cfg --output ./output --paths.train /content/train.spacy --paths.dev /content/dev.spacy

2023-12-03 07:28:06.687748: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 07:28:06.687821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 07:28:06.687866: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 07:28:10.334084: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
base_config.cfg
You can now add your data and train your pipeline:
python -m spacy train base_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
2023-12-03 07:28:20.600894: E

In [ ]:
nlp_ner = spacy.load("./output/model-best")
colors = {"IP": "#F08080", "MALWARE_NAME": "#EEE8AA", "DOMAIN_NAME":"#8FBC8B",
          "HASH_VAL":"#F9E79F", "ACTOR": "#F7DC6F", "HASH_VAL":"#F4D03F",
           "GEO_LOCATION":"#F8C471", "URL": "#F5B041"}
options = {"colors": colors}


In [ ]:
for example in test:
  doc = nlp_ner(example.get('text'))
  print("....Next batch .....")
  spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

....Next batch .....


....Next batch .....


....Next batch .....


....Next batch .....


/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


...

....Next batch .....


....Next batch .....


....Next batch .....


....Next batch .....


....Next batch .....


In [ ]:
import spacy
from spacy.scorer import Scorer
from spacy.training.example import Example

scorer = Scorer()
example = []
for obs in test:
    # print('Input for a prediction:', obs['text'])
    pred = nlp_ner(obs['text'])  ## custom_nlp is the custome model I am using to generate docs
    # print('Predicted based off of input:', pred, '// Entities being reviewed:', obs['entities'])
    temp = Example.from_dict(pred, {'entities': obs['entities']})
    example.append(temp)
scores = scorer.score_spans(example, "ents")


/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Escola Digital, a Brazilian website offering a wid..." with entities "[(0, 13, 'DOMAIN_NAME'), (18, 26, 'GEO_LOCATION'),...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "The JCPA website's homepage is infected with a mal..." with entities "[(164, 197, 'DOMAIN_NAME')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Follo

In [ ]:
test[0]

{'text': 'Escola Digital, a Brazilian website offering a wide range of digital courses for both students and teachers was found leaking the personal records of nearly 75,000 active users between 2016 and 2017. MyTopDog, a South African children-oriented study platform providing practice tests and interactive games, exposed over 800,000 student records, courtesy of a misconfigured Amazon S3 bucket. Okoo, an online learning platform for children in Kazakhstan, exposed 7,200 user records that included PII and nearly 1 million entries regarding user activity on the platform and analytics. Square Panda, a US-based virtual platform that helps children learn how to read and write, exposed the information of nearly 15,000 users. Playground Sessions, a virtual piano lesson platform in the United States, revealed the private information of around 4,100 users registered between 2011 and 2013.',
 'entities': [(0, 13, 'DOMAIN_NAME'),
  (18, 26, 'GEO_LOCATION'),
  (200, 207, 'DOMAIN_NAME'),
  (212, 

In [ ]:
def format_df(data):
  data_list = []
  for elem in data:
    for entity in elem['entities']:
      new_dict = {}
      new_dict['text'] = elem['text']
      new_dict['value'] = elem['text'][entity[0]: entity[1]]
      new_dict['label'] = entity[2]
      data_list.append(new_dict)
  return pd.DataFrame(data_list)


In [ ]:
test_df = format_df(test)

In [ ]:
test_df

,text,value,label
0,"Escola Digital, a Brazilian website offering a...",Escola Digita,DOMAIN_NAME
1,"Escola Digital, a Brazilian website offering a...",Brazilia,GEO_LOCATION
2,"Escola Digital, a Brazilian website offering a...",MyTopDo,DOMAIN_NAME
3,"Escola Digital, a Brazilian website offering a...",South Africa,GEO_LOCATION
4,"Escola Digital, a Brazilian website offering a...",Oko,DOMAIN_NAME
5,"Escola Digital, a Brazilian website offering a...",Kazakhsta,GEO_LOCATION
6,"Escola Digital, a Brazilian website offering a...",Square Pand,DOMAIN_NAME
7,"Escola Digital, a Brazilian website offering a...",US-base,GEO_LOCATION
8,"Escola Digital, a Brazilian website offering a...",Playground Session,DOMAIN_NAME
9,"Escola Digital, a Brazilian website offering a...",United State,GEO_LOCATION


In [ ]:
test_df.groupby(['label']).count()

,text,value
label,,
ACTOR,4,4
DOMAIN_NAME,11,11
FILE_PATH,3,3
GEO_LOCATION,16,16
HASH_VAL,1,1
MALWARE_NAME,9,9
URL,2,2


In [ ]:
scores

{'ents_p': 0.2916666666666667,
 'ents_r': 0.35,
 'ents_f': 0.31818181818181823,
 'ents_per_type': {'GEO_LOCATION': {'p': 0.3333333333333333,
   'r': 0.4,
   'f': 0.3636363636363636},
  'DOMAIN_NAME': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'MALWARE_NAME': {'p': 0.36363636363636365,
   'r': 0.5714285714285714,
   'f': 0.4444444444444444},
  'URL': {'p': 1.0, 'r': 1.0, 'f': 1.0},
  'ACTOR': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'HASH_VAL': {'p': 0.0, 'r': 0.0, 'f': 0.0},
  'FILE_PATH': {'p': 0.0, 'r': 0.0, 'f': 0.0}}}

In [ ]:
for element in training_data:
  if element.get('entities')[0] == (164, 197, 'DOMAIN_NAME'):
     spacy.displacy.render(nlp_ner(element.get('text')), style="ent", options= options, jupyter=True)


IndexError: ignored